In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path to your saved folder (local or Kaggle outputs)
SAVE_PATH = "D:\\ML_Project\\trained_model"

tokenizer = AutoTokenizer.from_pretrained(SAVE_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_PATH)
def translate(text, beams=6):
    inputs = tokenizer(text, return_tensors="pt")
    inputs.pop("token_type_ids", None)  # some tokenizers add this

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        num_beams=beams,
        do_sample=False,
        no_repeat_ngram_size=3,
        repetition_penalty=1.05,
        length_penalty=1.0,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# Garhwali → Hindi
print(translate("<gar> translate to <hin>: mi kitaab paddu chon"))

# Roman Garhwali → English
print(translate("<rgar> translate to <eng>: mi kitaab paddu chon"))



मैं किताब पढ़ रहा हूँ
I am reading a book.


In [4]:
import gradio as gr

def translate_ui(text, src_token, tgt_token):
    prompt = f"{src_token} translate to {tgt_token}: {text}"
    try:
        return translate(prompt)
    except Exception as e:
        return "Error"


with gr.Blocks() as demo:

    gr.HTML("""
    <style>

    /* Remove API + Settings + Footer */

    footer,
    .gradio-container footer,
    #footer,
    .app-footer,
    #built-with-gradio,
    .dark\\:bg-gray-800,
    .light\\:bg-gray-100,
    .bg-gray-200,
    #theme-toggle,
    #settings-btn,
    #account-menu,
    button[data-testid="api-button"],
    #api-button,
    button[data-testid="settings-button"],
    #settings-button,
    .svelte-drgfjc,
    .svelte-12s9m51,
    .svelte-1u9q1n2,
    div:has(> svg[data-icon="gradio"]),
    div[data-testid="footer"],
    .bottom-bar,
    .bottom-bar * {
        display: none !important;
        opacity: 0 !important;
        visibility: hidden !important;
    }

    /* Theme */

    body, .gradio-container {
        background: #52B2BF !important;
        font-weight: 800 !important;
        font-family: 'Segoe UI', sans-serif;
    }

    h1 {
        font-size: 34px !important;
        font-weight: 700 !important;
        color: #FFD700 !important;
        text-shadow: 0 0 4px black;
        text-align: center;
        margin-bottom: 20px;
    }

    .label {
        font-size: 18px !important;
        font-weight: bold !important;
        color: #FFD700 !important;
    }

    select {
        background: #D9D9D9 !important;
        color: black !important;
        border: none !important;
        border-radius: 6px !important;
        height: 45px !important;
        font-size: 18px !important;
        font-weight: 800 !important;
        padding-left: 10px !important;
    }

    textarea {
        background: #E6EEF1 !important;
        color: black !important;
        font-size: 16px !important;
        border-radius: 6px !important;
        padding: 12px !important;
        border: none !important;
    }

    /* Bigger Translate Button */

    button {
        background: #FF007F !important;
        color: white !important;
        font-weight: bold !important;
        border-radius: 8px !important;
        width: 220px !important;
        height: 60px !important;
        font-size: 22px !important;
        margin-top: 25px !important;
        border: none !important;
    }

    button:hover {
        background: #FF4FA0 !important;
    }

    /* Bigger gap between Input and Output */

    .gr-row > .gr-column:nth-child(1) {
        margin-right: 70px !important;
    }

    .gr-row > .gr-column:nth-child(2) {
        margin-left: 70px !important;
    }

    </style>
    """)

    gr.Markdown("<h1>🌐 Text Translator</h1>")

    with gr.Row():
        src_dropdown = gr.Dropdown(
            choices=["Garhwali"],
            label="Source Language",
            value="Garhwali"
        )

        gr.Markdown("<h2 style='text-align:center; font-size:36px; color:white;'>⇄</h2>")

        tgt_dropdown = gr.Dropdown(
            choices=["Hindi", "English"],
            label="Target Language",
            value="Hindi"
        )

    with gr.Row():
        input_text = gr.Textbox(
            lines=10,
            label="Input Text",
            placeholder="Enter text here..."
        )

        output_text = gr.Textbox(
            lines=10,
            label="Translated Output"
        )

    with gr.Row():
        translate_btn = gr.Button("Translate")

    def on_translate(text, src_ui, tgt_ui):
        src_token = "<rgar>" if src_ui == "Garhwali" else "<gar>"
        tgt_token = "<hin>" if tgt_ui == "Hindi" else "<eng>"
        return translate_ui(text, src_token, tgt_token)

    translate_btn.click(
        on_translate,
        inputs=[input_text, src_dropdown, tgt_dropdown],
        outputs=output_text
    )


demo.launch(share=True, inline=True)


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://1e3279ba561858bb43.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
